# Tutorial: Set-up, create, and train a High-Skip Network (HSN)

In this notebook, we will create and train a High Skip Network in the simplicial complex domain, as proposed in the paper by Hajij et. al: High Skip Networks: A Higher Order Generalization of Skip Connections (https://openreview.net/pdf?id=Sc8glB-k6e9). We will build a simple toy dataset from scratch using TopoNetX. We train the model to perform binary node classification. 

In [3]:
import torch
import numpy as np
from toponetx import SimplicialComplex
from topomodelx.nn.simplicial.hsn_layer import HSNLayer

# Pre-processing

## Create domain ##

The first step is to define the topological domain on which the TNN will operate, as well as the neighborhod structures characterizing this domain. We will only define the neighborhood matrices that we plan on using.

Here, we build a simple simplicial complex domain. TopoNetX is capable of defining a regular simplicial complex using only a set of given edges. In this case, we define two edges between three nodes. As such, the domain will be endowed with three nodes. We use the propoerty .simplices to list the cells of the domain.

In [4]:
edge_set = [[1, 2], [1, 3]]

domain = SimplicialComplex(edge_set)
domain.simplices

SimplexView([(1,), (2,), (3,), (1, 2), (1, 3)])

## Create neighborhood structures. ##

Now we retrieve the neighborhood structures (i.e. their representative matrices) that we will use to send messges on the domain. In this case, we need the boundary matrix (or incidence matrix) $B_1$ and the adjacency matrix $A_{\uparrow,0}$ on the nodes. For a santiy check, we show that the shape of the $B_1 = n_\text{nodes} \times n_\text{edges}$ and $A_{\uparrow,0} = n_\text{nodes} \times n_\text{nodes}$.

In [5]:
incidence_1 = domain.incidence_matrix(rank=1)
adjacency_0 = domain.adjacency_matrix(rank=0)

print("incidence_1\n", incidence_1.todense())
print("adjacency_0\n", adjacency_0.todense())

incidence_1
 [[-1. -1.]
 [ 1.  0.]
 [ 0.  1.]]
adjacency_0
 [[0. 1. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


Now we convert the neighborhood structures to torch tensors.

In [6]:
incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
adjacency_0 = torch.from_numpy(adjacency_0.todense()).to_sparse()

## Create signal ##

Since our task will be node classification, we must define an input signal (at least one datapoint) on the nodes. The signal will have shape $n_\text{nodes} \times$ in_channels, where in_channels is the dimension of each cell's feature. Here, we take in_channels = channels_nodes $ = 2$.

In [7]:
x_nodes = torch.tensor([[1.0, 1.0], [2.0, 2.0], [1.0, 1.0]])
channels_nodes = x_nodes.shape[-1]

# Create the Neural Network

Using the HSNLayer class, we create a neural network with stacked layers.

In [8]:
class HSN(torch.nn.Module):
    def __init__(self, channels, n_layers=2):
        super().__init__()
        layers = []
        for _ in range(n_layers):
            layers.append(
                HSNLayer(
                    channels=channels,
                )
            )
        self.linear = torch.nn.Linear(channels, 1)
        self.layers = layers

    def forward(self, x_0, incidence_1, adjacency_0):
        for layer in self.layers:
            x_0 = layer(x_0, incidence_1, adjacency_0)
        return self.linear(x_0)

# Train the Neural Network

We specify the model with our pre-made neighborhood structures, assign ground truth labels for the classification task, and specify an optimizer.

In [9]:
model = HSN(
    channels=channels_nodes,
    n_layers=2,
)
nodes_gt_labels = torch.Tensor([[0], [1], [1]])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

The following cell performs the training, looping over the network for 5 epochs.

In [10]:
for epoch in range(5):
    optimizer.zero_grad()
    nodes_pred_labels = model(x_nodes, incidence_1, adjacency_0)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        nodes_pred_labels, nodes_gt_labels
    )
    loss.backward()
    optimizer.step()